# 모바일 관련 분석
- 모바일 분석
- 갤럭시북 + 모바일 분석

In [1]:
#데이터셋 가져오기
#order 컬럼명 바꾸기 - 숫자로 되어 있는 부분에 Order (purchase event) 입력
import pandas as pd
#카테고리 전처리
import re
# 장바구니 분석
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import apriori, association_rules
import datetime as dt
import os
from collections import Counter

In [8]:
input_path = 'C:/GMG_R&D/3. 장바구니 분석/1. 한총' #input("작업 파일 경로를 입력하세요")
os.chdir(input_path)
path=os.getcwd()
print(path)

#폴더 생성
#x = dt.datetime.now()

file_name=f'모바일 갤럭시 분석 결과'

if not os.path.exists(file_name):
    os.mkdir(file_name) 

C:\GMG_R&D\3. 장바구니 분석\1. 한총


In [9]:
def frozen_convert(sets) : 
        return [list(sets)]

### 1. 한총 데이터 로드

In [5]:
df= pd.read_clipboard(sep='\t')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50001 entries, Model Code (v41) to EF-RS906CNEGKR,EP-TA800XWKGKR
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   2020년            50001 non-null  int64
 1   2021년            50001 non-null  int64
 2   2022.01~2022.11  50001 non-null  int64
dtypes: int64(3)
memory usage: 1.5+ MB


In [6]:
df=df[1:]
df=df.reset_index().rename(columns={"index": "Model Code (v41)"})
df['2020.01~2022.11']=df.iloc[:,1:4].sum(axis=1)
df

,Model Code (v41),2020년,2021년,2022.01~2022.11,2020.01~2022.11
0,EP-DG930MBKGKR,9746,6236,3665,19647
1,AA-BC7AU5B,9365,0,0,9365
2,SM-R175NZWAKOO,4961,344,0,5305
3,SM-N986NZNEKOO,4950,172,0,5122
4,SM-R175NZBAKOO,4906,127,0,5033
...,...,...,...,...,...
49995,"EJ-P5450SBEGKR,GP-TTT870KDATK,GP-TTT870KDATK",0,0,1,1
49996,"1FF10014,1FF10014,1FF10014,SAM20220221001,SAM2...",0,0,1,1
49997,"V00073,V00225,V00235",0,0,1,1
49998,"21,405,702,145,135",0,0,1,1


### 2. 카테고리

In [2]:
#카테고리 가져오기
rex= pd.read_clipboard(sep='\t')
rex_book=rex
rex.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Category1D        5 non-null      object
 1   Category2D        5 non-null      object
 2   Category2D_Value  5 non-null      object
dtypes: object(3)
memory usage: 248.0+ bytes


### 1) 비중

In [9]:

new_df=[]

for mobile in df.columns[1:]:
    print(f'************< {mobile} >******************')
    
    all=df[mobile].sum()
    one=df[~df[['Model Code (v41)',mobile]]['Model Code (v41)'].str.contains(',')][mobile].sum()
    two=df[df[['Model Code (v41)',mobile]]['Model Code (v41)'].str.contains(',')][mobile].sum()
    percent_one=round((df[~df[['Model Code (v41)',mobile]]['Model Code (v41)'].str.contains(',')][mobile].sum()/all)*100,2)
    percent_two=round((df[df[['Model Code (v41)',mobile]]['Model Code (v41)'].str.contains(',')][mobile].sum()/all)*100,2)

    
    
    print('전체: ',all)
    print('1개: ',one)
    print('2개이상: ',two)
    print("제품 1개 구매 비중: ",percent_one)
    print("제품 2개이상 구매 비중: ",percent_two)

    new_df1=[]
    for i in [all, one, two, percent_one,percent_two]:
        new_df1.append(i)
    new_df.append(new_df1)

# 데이터 저장3: 데이터 기준
pd.DataFrame(new_df,columns=['전체','1개','2개이상','제품 1개 구매 비중', '제품 2개이상 구매 비중'],
            index=[df.columns[1],df.columns[2],df.columns[3],df.columns[4]]).to_csv(f'{path}/{file_name}/[최종]기초통계량결과.csv', encoding='utf-8-sig')
pd.DataFrame(new_df,columns=['전체','1개','2개이상','제품 1개 구매 비중', '제품 2개이상 구매 비중'])

************< 2020년 >******************
전체:  365380
1개:  301555
2개이상:  63825
제품 1개 구매 비중:  82.53
제품 2개이상 구매 비중:  17.47
************< 2021년 >******************
전체:  738544
1개:  656707
2개이상:  81837
제품 1개 구매 비중:  88.92
제품 2개이상 구매 비중:  11.08
************< 2022.01~2022.11 >******************
전체:  842619
1개:  784183
2개이상:  58436
제품 1개 구매 비중:  93.06
제품 2개이상 구매 비중:  6.94
************< 2020.01~2022.11 >******************
전체:  1946543
1개:  1742445
2개이상:  204098
제품 1개 구매 비중:  89.51
제품 2개이상 구매 비중:  10.49


,전체,1개,2개이상,제품 1개 구매 비중,제품 2개이상 구매 비중
0,365380,301555,63825,82.53,17.47
1,738544,656707,81837,88.92,11.08
2,842619,784183,58436,93.06,6.94
3,1946543,1742445,204098,89.51,10.49


### 1) 모바일 분석

In [12]:
final_year={}

for mobile in df.columns[1:]:
    print(f'************< {mobile} >******************')

    mobile_df=df[df[mobile]!= 0]
    #print('order 0제외 행개수:',mobile_df.shape[0])
    mobile_df_1=mobile_df[~mobile_df[['Model Code (v41)',mobile]]['Model Code (v41)'].str.contains(',')]
    mobile_df_2=mobile_df[mobile_df[['Model Code (v41)',mobile]]['Model Code (v41)'].str.contains(',')]
    
    final_order={}
    code_book={'전체':mobile_df,'1개':mobile_df_1,'2개이상':mobile_df_2}
    for key, value in code_book.items():
        value=value.reset_index(drop=True)
        data=value
        a = data['Model Code (v41)'].tolist()
        b = ','.join(a) #리스트 문자열로
        c = b.split(',') #문자열 개별 문자로 리스트화 
        model = list(set(c)) #중복 제거

        #전체 SKU 담을 딕셔너리
        dic_category = {}

        for m in model :
            flag = 1
            for i in range(len(rex)):
                r = re.compile(rex.Category2D_Value[i])
                if r.search(m):
                    dic_category[m] = rex.Category1D[i] +'_'+ rex.Category2D[i]  #rex.Category1D[i] +'_'+  #sku 규칙을 알고자한다면, r.search(m).group()
                    flag = 0
                    break
            if flag :
                dic_category[m] = '기타'

        #데이터셋 만들기
        dataset = []    # 데이터셋 구조 [ [a,b], [a,c], [a,b,c] .... ]

        for i in range(len(data)) :
            sku = data['Model Code (v41)'][i]
            revised = sku.split(",")
            code = []
            for j in range(len(revised)) :
                code.append(dic_category[revised[j]])   #각 SKU 리스트에 담기
            
            for k in range(data[mobile][i]) :   #order 수만큼 담기
                dataset.append(code) 
        print(f'<<<{key}>>>')
        print('기존 dataset 행 수: ',len(dataset))

        #기타제거
        ho = []
        rm_set = {'기타'}
        for s in dataset:
            arr_new = [i for i in s if i not in rm_set]
            ho.append(arr_new) 
        dataset=ho
        
        print('기타 제거 후, 행 수: ',len(dataset))


        category='모바일'
        select_mobile=[s for s in dataset if any(category in l for l in s)]
        print('모바일이 포함된 행수: ',len(select_mobile))  
    

        per_order={}
        # 실제 구매 비중
        for PRODU in rex[rex['Category1D']=='모바일']['Category2D']:
            df_count=[]
            for s in select_mobile:
                for i in s:
                    if PRODU in i:
                        df_count.append(s)
            per_order[PRODU]=len(df_count)     
          

        #실제데이터중 UNIQE 값으로
        count=count=Counter([tuple(sorted(i)) for i in sorted([tuple(set(item)) for item in select_mobile])]) 
        da=sorted(count.items(), key=lambda i: i[1], reverse=True)
        #조합별 카운트 데이터 저장
        pd.DataFrame(da).to_csv(f'{path}/{file_name}/{category}_{key}_결과_{mobile}.csv', encoding='utf-8-sig')



        if  key=='2개이상':
            #장바구니 분석
            te = TransactionEncoder()
            te_ary = te.fit(select_mobile).transform(select_mobile) #True / False
            mobile2 = pd.DataFrame(te_ary, columns=te.columns_)
            itemsets_data = apriori(mobile2, min_support=0.01, use_colnames=True).sort_values('support', ascending=False).reset_index(drop=True)
            rules = association_rules(itemsets_data, metric='lift', min_threshold=1).sort_values('lift',ascending=False).reset_index(drop=True)
            rules['count']=(rules['support']*len(select_mobile)).astype(int)

            x = dt.datetime.now()

            num_conq=1
            #N=4


            for num_ant in [1,2,3]:
                final_mobile = rules[rules.apply(lambda x:True if len(x.antecedents) == int(num_ant) else False, axis=1) & rules.apply(lambda x:True if len(x.consequents) == int(num_conq) else False, axis=1)]
                final_mobile = final_mobile.sort_values('confidence', ascending=False)
                #final_mobile = final_mobile.round(N)
                final_mobile[['antecedents','consequents']] =final_mobile[['antecedents','consequents']].applymap(lambda x: frozen_convert(x))
                final_mobile['antecedents']=final_mobile['antecedents'].astype(str).apply(lambda x: x.replace('[[','['))
                final_mobile['antecedents']=final_mobile['antecedents'].apply(lambda x: x.replace(']]',']'))
                final_mobile['consequents']=final_mobile['consequents'].astype(str).apply(lambda x: x.replace('[[','['))
                final_mobile['consequents']=final_mobile['consequents'].apply(lambda x: x.replace(']]',']'))
                final_mobile.to_excel(f'{path}/{file_name}/{category}_{key}_한총 {mobile} 분석결과파일_{num_ant}.xlsx')
        final_order[key]=per_order
    final_year[mobile]=final_order

year_order=pd.DataFrame.from_dict(final_year, orient="index").stack().to_frame()
# 데이터 저장2: 모바일 항목 내에 카운트   
pd.DataFrame(year_order[0].values.tolist(), index=year_order.index).to_csv(f'{path}/{file_name}/[최종]{category}_전체 주문수 결과.csv', encoding='utf-8-sig')


************< 2020년 >******************
<<<전체>>>
기존 dataset 행 수:  365380
기타 제거 후, 행 수:  365380
모바일이 포함된 행수:  258098
<<<1개>>>
기존 dataset 행 수:  301555
기타 제거 후, 행 수:  301555
모바일이 포함된 행수:  202348
<<<2개이상>>>
기존 dataset 행 수:  63825
기타 제거 후, 행 수:  63825
모바일이 포함된 행수:  55750
************< 2021년 >******************
<<<전체>>>
기존 dataset 행 수:  738544
기타 제거 후, 행 수:  738544
모바일이 포함된 행수:  570758
<<<1개>>>
기존 dataset 행 수:  656707
기타 제거 후, 행 수:  656707
모바일이 포함된 행수:  502446
<<<2개이상>>>
기존 dataset 행 수:  81837
기타 제거 후, 행 수:  81837
모바일이 포함된 행수:  68312
************< 2022.01~2022.11 >******************
<<<전체>>>
기존 dataset 행 수:  842619
기타 제거 후, 행 수:  842619
모바일이 포함된 행수:  599892
<<<1개>>>
기존 dataset 행 수:  784183
기타 제거 후, 행 수:  784183
모바일이 포함된 행수:  574324
<<<2개이상>>>
기존 dataset 행 수:  58436
기타 제거 후, 행 수:  58436
모바일이 포함된 행수:  25568
************< 2020.01~2022.11 >******************
<<<전체>>>
기존 dataset 행 수:  1946543
기타 제거 후, 행 수:  1946543
모바일이 포함된 행수:  1428748
<<<1개>>>
기존 dataset 행 수:  1742445
기타 제거 후, 행 수:  1742445
모바일

### 2) 갤럭시북 + 모바일 분석

In [4]:
rex_book.loc[rex_book.shape[0]] = ['PC/주변기기', '갤럭시북', '^NT']
rex_book

,Category1D,Category2D,Category2D_Value
0,모바일,스마트폰,^SM-A|^SM-F|^SM-G|^SM-J|^SM-M|^SM-N|^SM-E|^SM-...
1,모바일,태블릿,^SM-P|^SM-T|^SM-X|^SM-W
2,모바일,버즈,^SM-R19|^SM-R170|^SM-R175|^SM-R177|^SM-R18|^SM...
3,모바일,워치,^SM-R[235789]*|^SM-R500
4,모바일,모바일 액세서리,^EB|^EE|^EF|^EJ|^EP|^ET|^EI-T|^TS-7T|^GP|^LABC...
5,PC/주변기기,갤럭시북,^NT


In [23]:
final_year={}

for mobile in df.columns[1:]:
    print(f'************< {mobile} >******************')

    mobile_df=df[df[mobile]!= 0]
    #print('order 0제외 행개수:',mobile_df.shape[0])
    mobile_df_1=mobile_df[~mobile_df[['Model Code (v41)',mobile]]['Model Code (v41)'].str.contains(',')]
    mobile_df_2=mobile_df[mobile_df[['Model Code (v41)',mobile]]['Model Code (v41)'].str.contains(',')]
    
    final_order={}
    code_book={'전체':mobile_df,'1개':mobile_df_1,'2개이상':mobile_df_2}
    for key, value in code_book.items():
        value=value.reset_index(drop=True)
        data=value
        a = data['Model Code (v41)'].tolist()
        b = ','.join(a) #리스트 문자열로
        c = b.split(',') #문자열 개별 문자로 리스트화 
        model = list(set(c)) #중복 제거

        #전체 SKU 담을 딕셔너리
        dic_category = {}

        for m in model :
            flag = 1
            for i in range(len(rex_book)):
                r = re.compile(rex_book.Category2D_Value[i])
                if r.search(m):
                    dic_category[m] = rex_book.Category1D[i] +'_'+ rex_book.Category2D[i]  #rex.Category1D[i] +'_'+  #sku 규칙을 알고자한다면, r.search(m).group()
                    flag = 0
                    break
            if flag :
                dic_category[m] = '기타'

        #데이터셋 만들기
        dataset = []    # 데이터셋 구조 [ [a,b], [a,c], [a,b,c] .... ]

        for i in range(len(data)) :
            sku = data['Model Code (v41)'][i]
            revised = sku.split(",")
            code = []
            for j in range(len(revised)) :
                code.append(dic_category[revised[j]])   #각 SKU 리스트에 담기
            
            for k in range(data[mobile][i]) :   #order 수만큼 담기
                dataset.append(code) 
        print(f'<<<{key}>>>')
        print('기존 dataset 행 수: ',len(dataset))

        #기타제거
        ho = []
        rm_set = {'기타'}
        for s in dataset:
            arr_new = [i for i in s if i not in rm_set]
            ho.append(arr_new) 
        dataset=ho
        
        print('기타 제거 후, 행 수: ',len(dataset))


        #####################   갤럭시북 추가 ###################################
      
        category1='갤럭시북'
        select_book=[s for s in dataset if any(category1 in l for l in s)]
        print('갤럭시북이 포함된 행수: ',len(select_book))  

        category='모바일'
        select_mobile=[s for s in select_book if any(category in l for l in s)]
        print('갤럭시북 + 모바일이 포함된 행수: ',len(select_mobile))  
    

        per_order={}
        # 실제 구매 비중  (갤럭시북+모바일의 경우 수치가 낮음)
        for PRODU in rex_book['Category2D']: #[rex['Category1D']=='모바일']['Category2D']:
            df_count=[]
            for s in dataset:
                for i in s:
                    if PRODU in i:
                        df_count.append(s)
            per_order[PRODU]=len(df_count)     
          

        #실제데이터중 UNIQE 값으로
        count=count=Counter([tuple(sorted(i)) for i in sorted([tuple(set(item)) for item in select_mobile])]) 
        da=sorted(count.items(), key=lambda i: i[1], reverse=True)
        #조합별 카운트 데이터 저장
        pd.DataFrame(da).to_csv(f'{path}/{file_name}/{category1}_{key}_결과_{mobile}.csv', encoding='utf-8-sig')



        if  key=='2개이상':
            #장바구니 분석
            te = TransactionEncoder()
            te_ary = te.fit(select_mobile).transform(select_mobile) #True / False
            mobile2 = pd.DataFrame(te_ary, columns=te.columns_)
            itemsets_data = apriori(mobile2, min_support=0.01, use_colnames=True).sort_values('support', ascending=False).reset_index(drop=True)
            rules = association_rules(itemsets_data, metric='lift', min_threshold=1).sort_values('lift',ascending=False).reset_index(drop=True)
            rules['count']=(rules['support']*len(select_mobile)).astype(int)

            x = dt.datetime.now()

            num_conq=1
            #N=4


            for num_ant in [1,2,3]:
                final_mobile = rules[rules.apply(lambda x:True if len(x.antecedents) == int(num_ant) else False, axis=1) & rules.apply(lambda x:True if len(x.consequents) == int(num_conq) else False, axis=1)]
                final_mobile = final_mobile.sort_values('confidence', ascending=False)
                #final_mobile = final_mobile.round(N)
                final_mobile[['antecedents','consequents']] =final_mobile[['antecedents','consequents']].applymap(lambda x: frozen_convert(x))
                final_mobile['antecedents']=final_mobile['antecedents'].astype(str).apply(lambda x: x.replace('[[','['))
                final_mobile['antecedents']=final_mobile['antecedents'].apply(lambda x: x.replace(']]',']'))
                final_mobile['consequents']=final_mobile['consequents'].astype(str).apply(lambda x: x.replace('[[','['))
                final_mobile['consequents']=final_mobile['consequents'].apply(lambda x: x.replace(']]',']'))
                final_mobile.to_excel(f'{path}/{file_name}/{category1}_{key}_한총 {mobile} 분석결과파일_{num_ant}.xlsx')
        final_order[key]=per_order
    final_year[mobile]=final_order

year_order=pd.DataFrame.from_dict(final_year, orient="index").stack().to_frame()
# 데이터 저장2: 모바일 항목 내에 카운트   
pd.DataFrame(year_order[0].values.tolist(), index=year_order.index).to_csv(f'{path}/{file_name}/[최종]{category1}_전체 주문수 결과.csv', encoding='utf-8-sig')


************< 2020년 >******************
<<<전체>>>
기존 dataset 행 수:  365380
기타 제거 후, 행 수:  365380
갤럭시북이 포함된 행수:  5123
갤럭시북 + 모바일이 포함된 행수:  14
<<<1개>>>
기존 dataset 행 수:  301555
기타 제거 후, 행 수:  301555
갤럭시북이 포함된 행수:  5088
갤럭시북 + 모바일이 포함된 행수:  0
<<<2개이상>>>
기존 dataset 행 수:  63825
기타 제거 후, 행 수:  63825
갤럭시북이 포함된 행수:  35
갤럭시북 + 모바일이 포함된 행수:  14
************< 2021년 >******************
<<<전체>>>
기존 dataset 행 수:  738544
기타 제거 후, 행 수:  738544
갤럭시북이 포함된 행수:  11997
갤럭시북 + 모바일이 포함된 행수:  22
<<<1개>>>
기존 dataset 행 수:  656707
기타 제거 후, 행 수:  656707
갤럭시북이 포함된 행수:  11730
갤럭시북 + 모바일이 포함된 행수:  0
<<<2개이상>>>
기존 dataset 행 수:  81837
기타 제거 후, 행 수:  81837
갤럭시북이 포함된 행수:  267
갤럭시북 + 모바일이 포함된 행수:  22
************< 2022.01~2022.11 >******************
<<<전체>>>
기존 dataset 행 수:  842619
기타 제거 후, 행 수:  842619
갤럭시북이 포함된 행수:  11617
갤럭시북 + 모바일이 포함된 행수:  36
<<<1개>>>
기존 dataset 행 수:  784183
기타 제거 후, 행 수:  784183
갤럭시북이 포함된 행수:  11299
갤럭시북 + 모바일이 포함된 행수:  0
<<<2개이상>>>
기존 dataset 행 수:  58436
기타 제거 후, 행 수:  58436
갤럭시북이 포함된 행수:  318
갤럭시북 +